# Multiple object tracking

In this note is meant to serve as a compendium of resources for multiple object tracking.
This mean, there will be less structured text (for the time being) and more links to resources, libraries, and repositories.
It should be looked at as a work-in-progress, with the intention to be expanded upon as more resources arise.


## Background
- Object detection vs. object tracking
    - Tracking is detection over multiple frames
    - Detection confidence can vary from frame to frame due to blur, pose, occlusion, etc.
    - One object may recieve many tracked ids if a detection is not produced in a (set of) frame(s)
    - Tracking requires then both detections but also a way to associate them across frames
    - Tracking can be online or offline, depending on the computational resources available
    - Modern tracking techniques typically use a detector (YOLO, ResNet, etc.) together with a tracker (SORT, ByteTrack, etc.)
- Movement predictions require info about velocity, direction, etc.
    - Can be constant, determined beforehand or learned from data
    - Can be adaptive, as seen in Kalman filters 
- Some classic methods exploit derivative-like estimatations of postion from changes in detections between frames
    - Even simplier ones just look at IOU between objects found through filtering
- Some trackers that leverage deep learning incorporate visual information about each detected object to help performance
    - This can be done through embeddings, which are learned representations of the object's appearance
    - These embeddings can be used to associate objects across frames, even when they are occluded or change appearance
- Other important topics
    - Augmentation techniques
        - Augment images together with labels to improve tracking performance
        - Helpful when training on a small or unbalanced dataset
        - Can include flipping, rotation, scaling, etc., but label must be updated accordingly
    - Evaluation
        - The MOTChallenges (MOT20, MOT17, MOT16) are popular benchmarks for evaluating multiple object tracking algorithms
        - Some other benchmarks include DanceTrack, SportsMOT, and WildTrack
        - Papers with Code has a leader board for the current state-of-the-art trackers on popular benchmarks here: https://paperswithcode.com/task/multi-object-tracking 

### Object detection

<br>
<img src="../img/OD_dance.png" width="750">
<br>
<i>Figure 1: Example of object detection of a frame from a dance video (<a href="https://github.com/DanceTrack/DanceTrack">Dataset</a>, <a href="https://github.com/noahcao/OC_SORT">Detector</a>)</i>

In object detection, a single frame is considered in isolation, and the goal is to identify and classify particular objects in the frame by drawing bounding boxes around them.
This can be done with a focus on a single object or multiple objects, depending on model objective, complexity, and available compute resources.
Detections are usually thresholded by a confidence score, and [non-maximum suppression](https://learnopencv.com/non-maximum-suppression-theory-and-implementation-in-pytorch/) is applied to remove overlapping boxes.
To achieve state-of-the-art performance on object detection alone, new architectures will likely be focused more on prediciton correctness rather than speed.

Some of the most polular object detection models include YOLO (maintained by [Ultralytics](https://github.com/ultralytics/ultralytics)) and Faster R-CNN.
A state-of-the-art leader board for object detection models can be found on the Papers with Code website [here](https://paperswithcode.com/task/object-detection).
There, you can find the most recent models, along with their whitepapers and code implementations.

### Object tracking
<br>
<img src="https://github.com/noahcao/OC_SORT/raw/master/assets/dancetrack0088_slow.gif" width="750">
<br>
<i>Figure 1: Example of object tracking in a dance video. Source: <a href="https://github.com/noahcao/OC_SORT">github.com/noahcao/OC_SORT</a></i>


Object tracking builds off object detection, but it considers multiple frames in a video sequence.
The goal is to associate detections across frames to track the same object over time.
In these cases, speed is often more important than initial correctness, as the model must process many frames in a video sequence.
The model must consider the object's movement, occlusion, and appearance changes, and update it's detection and tracking accordingly.

Typically, it is useful for these models to assign a unique ID to each object in the video sequence.
A good tracker will be able to maintain the same ID for an object even when it is occluded or changes appearance.
For more rudimentary tracking methods, re-identifying tracked objects occluded by other objects can be a challenge. 
This may result in the same object receiving new IDs after reappearance, or the object being lost entirely.

Modern trackers exploit deep learning components to include more temporal or visual information about the detections to assist in this process.
These more complex trackers can be slower or require more compute resources, introducing a trade-off between speed and correctness.
Decisions around this trade-off will depend on the specific use case and available resources.


## Tracking 

With a brief overview of object tracking as a task, we can now explore some techniques in more detail. 

### Classic
Tradional computer vision techniques can be applied to videos as light-weight trackers.
Below are a list of some of the most popular classic tracking algorithms, along with breif explanations and links to blog posts explaining further.
These simplified set-ups will require two main steps: object segementation and object association.



#### Segmentation



##### Background detection 
While not directly a tracking algorithm, background detection can be used in tandem with other traditional computer vision techniques to make up a light-weight tracker. 
The general idea is to determine a static background from a set of frames, to differentiate these pixels from moving objects.

A video's background can be determied as the [median pixel values](https://learnopencv.com/simple-background-estimation-in-videos-using-opencv-c-python/) for all frames, a process known as _temporal median filtering._
Such a technique assumes few moving objects and a static background, where moving objects are present in any given pixel in less than 50% of the frames.
This method is rudimentary but can be effective in simple tracking tasks, video surveillance, and other applications where the background is relatively static.

More details on background estimation can be found in this blogpost on [Simple Background Estimation in Videos using OpenCV](https://learnopencv.com/simple-background-estimation-in-videos-using-opencv-c-python/).

##### Contour detection
Before the wide-spread use of neural networks, one way of detecting edges in an image was to use greyscales and binary thresholding.
An input image could be converted to greyscale, removing any superfluous color information.
Then, a binary threshold is applied to the greyed image, where pixel values above a certain threshold are set to 1, and those below are set to 0.
The result was then a binary image, where edges were represented by the transition from 0 to 1. 
Here, a simple contour detection algorithm can be applied to trace the pixels along the edges of objects in the image.

![image](https://learnopencv.com/wp-content/uploads/2024/01/contour_asset.gif) 

_Example of contour detection in OpenCV. Source: [learnopencv.com](https://learnopencv.com/moving-object-detection-with-opencv/)_

This simple yet useful approach can be applied to videos with both static and dynamic background. 
The setup requires minimal computational resources and can therefore can be implemented in various stages of a pipelines. 
In preprocessing steps, contour detection can assist in object detection, telling a model where in a frame it should focus most, increasing detection and segmentation performance. 
Additionally, contour detection can help in post-detection stages to assist in association and tracking, when combined with IOU or other metrics.

A very simple tracker could be build by using contour detection to segment objects in a frame, labeling each segmented object with an id.
The "detection" step can be quickly run for each frame, recalculating the detections and labels for each frame. 
However, in videos where new objects appear or objects disappear, these labels will differ, thus requiring more complex logical around label association between frames. 
This topic will be explored more thoroughly in the section titled [Association](#Association).

A detailed walk-through implementing contour detection can be found [here](https://learnopencv.com/contour-detection-using-opencv-python-c/). 


##### Mean-shift clustering/segmentation
Similar to K-means clustering, mean-shift clustering is a method for finding the modes (or segmentations) of a distribution of data points.

<img src="../img/mean-shift.png" width="750"><br>

_Example of mean-shift clustering in a 2D feature space. Source: [Stanford Vision and Learning Lab](http://vision.stanford.edu/teaching/cs131_fall1920/slides/09_kmeans_mean_shift.pdf)_



The algorithm starts by [tessellating](https://dictionary.cambridge.org/dictionary/english/tessellate) the feature space with a grid of windows, each centered on a pixel (i.e. odd numbered height and width window sizes).
The algorithm then shifts each window to the mean of the data points, based on the selected feature space. 
Eventually, the windows will converge around a single point, which is considered a mode or peak in the data distribution.
Windows that converge near the same peak are then merged, and the process is repeated until all windows have converged.

In a simplfied 1D set-up (i.e. a greyscale image), the feature space could be the pixel intesity, between 0 and 255.
For colored image segmentation, one could instead use the RGB values of an input image. 
For an even more complex set-up, the feature space could be a combination of pixel color, texture, or other means of describing a subregion of an image.

Similar to contour detection, mean-shift clustering can be used in both pre- and post-processing stages of a pipeline, helping extract the mos timportant features of an input image.
A great resource for understanding both mean-shift filtering and k-means filtering (skipped in our note) can be found in this lecture from [Stanford Vision and Learning Lab](http://vision.stanford.edu/teaching/cs131_fall1920/slides/09_kmeans_mean_shift.pdf). 

<!-- 
#### Particle filter
- Use a set of particles to represent the object's position
- Update the particles based on the object's movement
- Can be effective for simple tracking tasks -->




#### Association




#### Kalman filter (SORT)
While not exactly an association technique by itself, Kalman filters are often used together with association funcitons like the Hungarian algorithm to update estimations about an object's position.
The Kalman filter is a recursive algorithm that estimates the state of a linear dynamic system from a series of noisy measurements.
It is used in many fields, including robotics, computer vision, and economics, to predict the future state of a system based on its past states.

The Kalman filter is based on two main assumptions:
<ol type="i">
    <li>The state of the system can be represented as a linear dynamic system.</li>
    <li>The noise in the system is Gaussian, representable via a mean about 0 and an unknown variance.</li>
</ol>

By a linear dynamic system, we mean that the state of the system at the next time step can be predicted based on the previous state and the system's dynamics (velocity and direction).
The step-functions are considered constant, and the system's estimated state can be predicted based on these constants, similar to a derivative-based physics simulation using $\frac{\delta x}{\delta t}$. 



<!-- It can be helpful to think recall your high-school physics classes, where you learned about classical mechanics to predict an object's future position given an initial position, intial velocity, and an acceleration:
$$
x(t) = x_0 + v_0t + \frac{1}{2}at^2 \quad \text{and} \quad  v(t) = v_0 + at \tag{1}
$$

Instead of the fixed, simple equations above, the Kalman filters take into account uncertainty around the system's estimates and measurements, and update the state estimate based on new measurements. -->

The Kalman filtering can be broken down into 3 main stages:
1. **Estimatation:** Estimate the current state of the system based on the fixed velocity and (randomly initialized) direction $\rightarrow \hat x_{t+1}$ 
2. **Detection:** Make a (noisy) measurement of the system $\rightarrow y_{t+1}$ 
3. **Update:** Update the state estimate with information of the new measurement, giving a more accurate prediction of the system's state $\rightarrow x_{t+1}$ 


The equations used for Kalman filters are as followed:

$$
x_{t+1} = Fx_t + u \quad \text{where} \quad  u \sim N(0, P) \tag{1}
$$

$$
y_{t+1} = Hx_t + v \quad \text{where} \quad  v \sim N(0, Q) \tag{2}
$$

Here, $F$ and $H$ are the constant state transition and observation matrices, respectively. 
The $u$ and $v$ terms are the process and measurement noise, respectively, with assumed means about 0 and variances $P$ and $Q$.

The true state of a system at time $t$ will always be unknown, but the most accuracte prediction is represented as $x_t$. 
$y_t$ is the measurement made at time $t$, which in our case is the outputs from our detection model.  
The approximation of the state at time $t$ given the transition matrix is $\hat x_t$. 
We can rewrite the equations above representing their respective approximations as:
$$
\hat x_{t+1} = F \hat x_t + u \quad \text{where} \quad u \sim N(0, P) \tag{3}
$$

$$
\hat y_{t+1} = H \hat x_t + v \quad \text{where} \quad  v \sim N(0, Q) \tag{4}
$$

The approximation $\hat x_{t+1}$ is then used to produce an estimated measurement $\hat y_{t+1}$, which is then used to produce the best prediction possible, $x_{t+1}$, starting the process over again.
The equaiton for updating the state estimate with the new measurement is as follows:
$$
x_{t+1} = \hat x_{t+1} + K(y_{t+1} - \hat y_{t+1}) \tag{5}
$$

Here $K$ is the Kalman ratio, which balances how much information from the the new measurement we want to update our state estimate with, based on the system's uncertianties.

One of the reasons Kalman filters are so useful is because they make use of _all_ the information available to the system.
The more information available to a system, the less uncertainty, and thus the more accurate the predictions.

Kalman filtering is often used in tracking applications to predict the object's position based on its velocity and update the position based on the detection.
The Simple Online Realtime Tracking (SORT) algorithm is a popular tracking algorithm that uses Kalman filtering for predictions and the Hungarian algorithm for association.

A really great resource for understanding Kalman filters is this [YouTube video](https://www.youtube.com/watch?v=IFeCIbljreY) by [Visually Explained](https://www.youtube.com/@VisuallyExplained).
A hand-written note covering some of these topics can also be found in this repository [here](../img/Kalman-Filtering-handwritten.pdf).


### Hungarian algorithm (SORT)
- Find the optimal assignment of objects across frames
- Can be effective for simple tracking tasks
- Can be combined with other methods for improved performance



#### Modern
- DeepSORT
    - https://github.com/nwojke/deep_sort/tree/master 
    - replace IOU with Mahalanobis distance
    - use embeddings to associate objects across frames
- StrongSORT
    - ...?
- OC_SORT
    - ...?
- ByteTrack
    - https://roboflow.com/how-to-track/yolov5 
    - ...?
- Nofair
    - https://github.com/tryolabs/norfair/blob/master/demos/yolov5/src/demo.py 
    - ...?